In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tts.datasets.ljspeech_dataset import LJSpeechDataset

dataset = LJSpeechDataset(
    dataset_path='data/LJSpeech-1.1', is_train=True, train_size=1.0, text_cleaners=['english_cleaners'],
    mel_spec_path='mels', alignment_path='alignments', sr=16000
)

In [3]:
batch = dataset[2]

In [4]:
import torch
torch.nn.functional.pad(dataset[2]['mel_target'], pad=(0, 900), mode='constant', value=0).size()

torch.Size([80, 1733])

In [5]:
dataset[2]['mel_target'].size()

torch.Size([80, 833])

In [6]:
batch['text'] = batch['text'].unsqueeze(0)
batch['duration'] = batch['duration'].unsqueeze(0)
batch['mel_target'] = batch['mel_target'].unsqueeze(0)

In [7]:
batch

{'raw_text': 'For although the Chinese took impressions from wood blocks engraved in relief for centuries before the woodcutters of the Netherlands, by a similar proces',
 'text': tensor([[43, 52, 55, 11, 38, 49, 57, 45, 52, 58, 44, 45, 11, 57, 45, 42, 11, 40,
          45, 46, 51, 42, 56, 42, 11, 57, 52, 52, 48, 11, 46, 50, 53, 55, 42, 56,
          56, 46, 52, 51, 56, 11, 43, 55, 52, 50, 11, 60, 52, 52, 41, 11, 39, 49,
          52, 40, 48, 56, 11, 42, 51, 44, 55, 38, 59, 42, 41, 11, 46, 51, 11, 55,
          42, 49, 46, 42, 43, 11, 43, 52, 55, 11, 40, 42, 51, 57, 58, 55, 46, 42,
          56, 11, 39, 42, 43, 52, 55, 42, 11, 57, 45, 42, 11, 60, 52, 52, 41, 40,
          58, 57, 57, 42, 55, 56, 11, 52, 43, 11, 57, 45, 42, 11, 51, 42, 57, 45,
          42, 55, 49, 38, 51, 41, 56,  6, 11, 39, 62, 11, 38, 11, 56, 46, 50, 46,
          49, 38, 55, 11, 53, 55, 52, 40, 42, 56]]),
 'duration': tensor([[ 1,  4,  8,  4, 10,  1,  2,  1, 11,  0,  0,  4,  0,  3,  0,  4,  2,  5,
           4, 15, 

In [8]:
batch['mel_target'].size()

torch.Size([1, 80, 833])

In [9]:
from tts.model.fastspeech1 import FastSpeechV1
from tts.text.symbols import symbols

model = FastSpeechV1(
    max_len=5000, vocab_size=len(symbols), pad_idx=0, n_blocks=3, n_heads=2, fft_kernel=3, lr_kernel=3, embed_dim=32, n_mels=80 
)

/home/dzhunk/University/dl-audio/tts/.venv/lib/python3.8/site-packages/torch/serialization.py:1101: SourceChangeWarning: source code of class 'glow.WaveGlow' has changed. Saved a reverse patch to WaveGlow.patch. Run `patch -p0 < WaveGlow.patch` to revert your changes.
  warnings.warn(msg, SourceChangeWarning)
/home/dzhunk/University/dl-audio/tts/.venv/lib/python3.8/site-packages/torch/serialization.py:1101: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. Tried to save a patch, but couldn't create a writable file ConvTranspose1d.patch. Make sure it doesn't exist and your working directory is writable.
  warnings.warn(msg, SourceChangeWarning)
/home/dzhunk/University/dl-audio/tts/.venv/lib/python3.8/site-packages/torch/serialization.py:1101: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. Saved a reverse patch to ModuleList.patch. Run `patch -p0 < ModuleList.patch` to revert your changes.
  warn

In [10]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
batch['text'] = batch['text'].to(device)
batch['duration'] = batch['duration'].to(device)
batch['mel_target'] = batch['mel_target'].to(device)
model = model.to(device)

In [12]:
o=model(**batch)

In [14]:
o['pred_mel'].size()

torch.Size([1, 819, 80])

In [15]:
batch['duration'].size()

torch.Size([1, 154])

In [16]:
model.text2voice(text='Demo speech.', dataset=dataset)

/home/dzhunk/University/dl-audio/tts/glow.py:270: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  audio = torch.cuda.FloatTensor(spect.size(0),


array([  2043,  32334,  10208, ..., -18178,  -8104,  27914], dtype=int16)

In [4]:
import scipy

sr, data = scipy.io.wavfile.read('/home/dzhunk/University/dl-audio/tts/data/LJSpeech-1.1/wavs/LJ001-0001.wav')